## Initial steps

Import libraries

In [33]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
training = pd.read_pickle("../Datasets/encoded_train.pkl")

In [ ]:
training.head()

In [ ]:
#training.info(1000)

## Training dataset preparation

In [ ]:
from sklearn.model_selection import train_test_split

y = training['notified']
X = training.drop('notified', axis=1)
test_size = 0.3
seed = 1011

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed )

## AdaBoost Clasifier

In [27]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

model =  AdaBoostClassifier(RandomForestClassifier(n_estimators=2000, max_depth=85, random_state=0), algorithm="SAMME.R", n_estimators=100, learning_rate=0.5)
model.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=85, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=2000, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False),
          learning_rate=0.5, n_estimators=100, random_state=None)

In [28]:
predictions = model.predict_proba(X_test)

### Evaluation

In [29]:
from sklearn import metrics
metrics.roc_auc_score(y_test, predictions[:,1])

0.9223118714721631

In [ ]:
0.9161883406684672
0.9217884795522725
0.9223118714721631

## Parameter tuning

### Grid search

In [37]:
from sklearn.model_selection import GridSearchCV

rf_n_estimators = list(range(1000,4000,500))
rf_max_depth = list(range(30,60,10))
ada_n_estimators = list(range(1,4))

hyperF = dict(base_estimator__n_estimators = rf_n_estimators, 
              base_estimator__max_depth = rf_max_depth,
              n_estimators = ada_n_estimators)

rf = RandomForestClassifier(random_state=0)
model = AdaBoostClassifier(base_estimator = rf, algorithm="SAMME.R")
gridF = GridSearchCV(model, hyperF, cv = 3, verbose = 6, 
                      n_jobs = 2, scoring='roc_auc', refit=True)
bestF = gridF.fit(X_train,y_train)

Fitting 3 folds for each of 54 candidates, totalling 162 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=2)]: Done   9 tasks      | elapsed:  2.9min
/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memor

In [38]:
print("Best score: ",bestF.best_score_)
print("Best params: ", bestF.best_params_)

Best score:  0.9158716599318227
Best params:  {'base_estimator__max_depth': 40, 'base_estimator__n_estimators': 2000, 'n_estimators': 2}


In [39]:
predictions = bestF.predict_proba(X_test)
metrics.roc_auc_score(y_test, predictions[:,1])

0.922233897546644